In [56]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [57]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM,Dense ,GRU,SimpleRNN,Conv1D,MaxPooling1D,Flatten , Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

from torch import optim
import argparse
import os
import pickle
import shutil
from random import random
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
from torch.nn import functional as F

In [58]:
def PrePadding(data, seglength):
    pad = (-len(data)) % seglength
    l = np.concatenate((data,np.zeros((pad, len(data[0])))))
    return l

In [59]:
def to_gpu(x, on_cpu=False, gpu_id=None):

    if torch.cuda.is_available() and not on_cpu:
        x = x.cuda(gpu_id)
    return x

In [60]:
ARDF = pd.read_csv(r"C:\Users\admin\Desktop\test excel\LB.csv", header= None, skiprows=2)
AR = np.array(ARDF)
print(f"raw data shape: {np.shape(AR)}")
seglength = 36000
newar = PrePadding(AR, seglength)
IGEsegs = np.array(np.split(newar, seglength)).reshape(-1, seglength, 20)
print(f"segment list shape:{np.shape(IGEsegs)}")
IGE_labels = np.ones(np.shape(IGEsegs)[0]).reshape(-1)


raw data shape: (1881497, 20)
segment list shape:(53, 36000, 20)


In [61]:
ARDF = pd.read_csv(r"C:\Users\admin\Desktop\test excel\pnes.csv", header= None, skiprows=2)
AR = np.array(ARDF)
print(f"raw data shape: {np.shape(AR)}")
seglength = 36000
newar = PrePadding(AR, seglength)
PNESsegs = np.array(np.split(newar, seglength)).reshape(-1, seglength, 20)
print(f"segment list shape:{np.shape(PNESsegs)}")
PNES_labels = np.zeros(np.shape(PNESsegs)[0]).reshape(-1)


raw data shape: (1964747, 20)
segment list shape:(55, 36000, 20)


In [62]:
x = np.append(IGEsegs, PNESsegs, axis=0)
y = np.append(IGE_labels, PNES_labels)
xx = to_gpu(torch.from_numpy(x))
yy = to_gpu(torch.from_numpy(y))

In [66]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.output_size = 1

        rnn = nn.LSTM
        self.eeg_rnn = rnn(2548, 256, bidirectional=False)
        self.eeg_rnn.add_module('eeg_rnn_dropout', nn.Dropout(0.2))
        self.eeg_rnn.add_module('eeg_rnn', rnn(256, 256,bidirectional=False))
        self.eeg_rnn.add_module('eeg_rnn_dropout2', nn.Dropout(0.2))

        self.project = nn.Sequential()
        self.project.add_module('project', 
                nn.Linear(in_features=256, out_features=self.output_size))
        self.project.add_module('project_activation', 
                                                nn.ReLU())
        self.project.add_module('project_softmax', nn.Softmax())
    
    def forward(self, eeg_samples):
        batch_size=eeg_samples.size(1)
        h1, (h2, _) = self.eeg_rnn(eeg_samples)
        eeg_feats = self.project(h1.view(batch_size,-1))
        return eeg_feats

In [65]:
xx.size(1)

36000